In [10]:
#importar algunas librerias que estaremos usando comunmente.
import numpy as np
import pandas as pd

In [16]:
class data_analyst:
    def __init__(self , file_name):
        self.file_name = file_name # Que sera el atributo, conocido con el nombre del archivo
        self.data =  None #Al instanciarlo, le cargaremos los datos respectivos con la libreria Pandas
    
    #De aqui en adelante son los metods
    
    def open_file(self):
        
        """
        Function to open files directly with the name, to reduce cost on our code
        """
        try:
            if self.file_name.endswith('.csv'):
                self.data = pd.read_csv(self.file_name)
                #print(f"Your .csv file was opened sucessfully")
            elif self.file_name.endswith('.xls') or self.file_name.endswith('.xlsx'):
                self.data = pd.read_excel(self.file_name)
                #print(f'Your .xls / xlsx was opened sucessfully')
            else:
                print('This file is not valid, please try with one with the ext csv / xls / xlsx ')
                
        except Exception as e:
            print(f"There is a error trying to open the file, error-code: {e}")
    
    def show_rows(self, rows = 10):
        """
        Funcion para mostrar un cabezal, o una cantidad de filas, por defecto esta en 10
        Args:
            rows (int, optional): _description_. Defaults to 10.
        """
        
        #iniciamos con una validacion de que nuestros datos tienen un valor asignado, dado que None es la representacion de la ausencia de valor.
        if self.data is not None:
            #Retornamos la impresion de las filas, con n valor, que en caos de que no se modifique rows = 10
            return self.data.head(rows)
        
        else:
            print(f'There is not a file upload yet')
    
    def info(self):
        """Funcion para retornar la funcion info de pandas, con el data_set que cargamos"""
        if self.data is not None:
            return self.data.info()
        
        else:
            print(f"There is not file upload yet")
    
    def describe(self):
        
        """Funcion para retornar una descripcion de pandas"""
        if self.data is not None:
            return self.data.describe()
        
        else:
            print("There is not file upload yet")
    
    def show_columns_names(self):
        """ Muestra y retorna el nombre de las columnas, las retorna en una lista-"""
        cols = [] # Se crea un lista para guardar el nombre de las columnas, la retornaremos en caso de querer usarla
        if self.data is not None:
            print("There is the name of the columns")
            for col in self.data.columns:
                print(col)
                cols.append(col)
        else:
            print('No hay columas en tu data_frame')
        
        return cols
    
    def rename_columns(self, actual_name, new_name):
        
        #Comenzamos validando la existencia de datos en nuestro data frame
        if self.data is not None:
            
            if actual_name in self.data.columns:
                self.data.rename(columns={actual_name: new_name}, inplace = True)
                print(f'The column "{actual_name}" was renamed as "{new_name}"')
            else:
                print(f'There is not a colum named as "{actual_name}"')
        else:
            print(f"There is file/ dataframe upload yet")
    
    def unique_values(self):
        
        if self.data is not None:  
            # Dictionary Comprehension para obtener el conteo de valores únicos en cada columna.
            # - 'col' es el nombre de la columna actual en 'self.data.columns'.
            # - La clave del diccionario es 'col'.
            # - El valor es el número de elementos únicos en la columna, obtenido con 'self.data[col].nunique()'.
            #usare un dict comprehencion, para practicar esta estructura
            unique_values = {col : self.data[col].nunique() for col in self.data.columns}
            
            # Iterando sobre el diccionario 'unique_values'.
            # - 'col' representa el nombre de la columna actual.
            # - 'count' representa el número de valores únicos de esa columna.
            # Utilizamos f-strings para imprimir de manera ordenada la columna y su número de valores únicos.
            for col, count in unique_values.items():
                print(f"Columna: {col} -> Valores Únicos: {count}")
        else:
            print(f'There is not file upload yet') 
            
#Creacion de proyectos
    

In [21]:
# En primera forma lo instanciamos
Oppenhim_File = data_analyst("imdb_oppenhimmer_Uncleaned.csv") # Instanciacion de objeto open

Oppenhim_File.open_file()

Oppenhim_File.unique_values()

columns_names = Oppenhim_File.show_columns_names()

Columna: 9/10
Murphy is exceptional
Orlando_Gardner19 July 2023
You'll have to have your wits about you and your brain fully switched on watching Oppenheimer as it could easily get away from a nonattentive viewer. This is intelligent filmmaking which shows it's audience great respect. It fires dialogue packed with information at a relentless pace and jumps to very different times in Oppenheimer's life continuously through it's 3 hour runtime. There are visual clues to guide the viewer through these times but again you'll have to get to grips with these quite quickly. This relentlessness helps to express the urgency with which the US attacked it's chase for the atomic bomb before Germany could do the same. An absolute career best performance from (the consistenly brilliant) Cillian Murphy anchors the film. This is a nailed on Oscar performance. In fact the whole cast are fantastic (apart maybe for the sometimes overwrought Emily Blunt performance). RDJ is also particularly brilliant in 

In [23]:
#Un nombre bastante grande para mi gusto
print(columns_names[0])


9/10
Murphy is exceptional
Orlando_Gardner19 July 2023
You'll have to have your wits about you and your brain fully switched on watching Oppenheimer as it could easily get away from a nonattentive viewer. This is intelligent filmmaking which shows it's audience great respect. It fires dialogue packed with information at a relentless pace and jumps to very different times in Oppenheimer's life continuously through it's 3 hour runtime. There are visual clues to guide the viewer through these times but again you'll have to get to grips with these quite quickly. This relentlessness helps to express the urgency with which the US attacked it's chase for the atomic bomb before Germany could do the same. An absolute career best performance from (the consistenly brilliant) Cillian Murphy anchors the film. This is a nailed on Oscar performance. In fact the whole cast are fantastic (apart maybe for the sometimes overwrought Emily Blunt performance). RDJ is also particularly brilliant in a return 

In [24]:
Oppenhim_File.rename_columns(columns_names[0], "Qualifity")

The column "9/10
Murphy is exceptional
Orlando_Gardner19 July 2023
You'll have to have your wits about you and your brain fully switched on watching Oppenheimer as it could easily get away from a nonattentive viewer. This is intelligent filmmaking which shows it's audience great respect. It fires dialogue packed with information at a relentless pace and jumps to very different times in Oppenheimer's life continuously through it's 3 hour runtime. There are visual clues to guide the viewer through these times but again you'll have to get to grips with these quite quickly. This relentlessness helps to express the urgency with which the US attacked it's chase for the atomic bomb before Germany could do the same. An absolute career best performance from (the consistenly brilliant) Cillian Murphy anchors the film. This is a nailed on Oscar performance. In fact the whole cast are fantastic (apart maybe for the sometimes overwrought Emily Blunt performance). RDJ is also particularly brilliant 

In [26]:
Oppenhim_File.show_rows()

,Qualifity
0,"8/10\nA challenging watch to be sure, but a wo..."
1,10/10\nA brilliantly layered examination of a ...
2,10/10\nNolan delivers a powerfull biopic that ...
3,10/10\nA Masterpiece\nmohameddawoud-2601919 Ju...
4,10/10\nAnd the Oscar goes to...\nmark-217-3070...
5,10/10\nAnother Cinematic Masterpiece by Christ...
6,8/10\nThis movie is the bomb!\nzeki-420 July 2...
7,9/10\nExceptional storytelling and Genius Cina...
8,10/10\nFUTURE CLASSIC and EXTREMELY IMPORTANT ...
9,8/10\nIs it just me ?\npdean-4913221 July 2023...
